In [110]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
from torch.utils.data import DataLoader
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import torchvision
import glob
import cv2

In [111]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [112]:
print(device)

cpu


In [113]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [114]:
#Dataloader

#Path for training and testing directory
train_path='C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1'
pred_path='C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
pred_loader=DataLoader(
    torchvision.datasets.ImageFolder(pred_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [115]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [116]:
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [125]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)
        output=self.conv2(output)
        output=self.relu2(output)
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            

In [133]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(pred_path+'/**/*.jpg'))

In [135]:
print(train_count,test_count)

14034 14034


In [136]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=6)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=6, bias=True)
)

In [137]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [128]:
#prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    index=output.data.numpy().argmax()
    pred=classes[index]
    return pred
    

In [142]:
images_path=glob.glob(pred_path+'/**/*.jpg')

In [143]:
images_path

['C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1\\buildings\\0.jpg',
 'C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1\\buildings\\10006.jpg',
 'C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1\\buildings\\1001.jpg',
 'C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1\\buildings\\10014.jpg',
 'C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1\\buildings\\10018.jpg',
 'C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1\\buildings\\10029.jpg',
 'C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1\\buildings\\10032.jpg',
 'C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1\\buildings\\10056.jpg',
 'C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1\\buildings\\1009.jpg',
 'C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1\\buildings\\10113.jpg',
 'C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1\\buildings\\1012.jpg',
 'C:/Users/Akanksha/OneDrive/Desktop/SCENE_DATA/Dataset1\\buildings\\10126.jpg',
 'C:/Users/Akanksha/OneDrive/Deskto

In [154]:
pred_dict={}

for i in images_path:
   pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [155]:
pred_dict

{'Dataset1\\buildings\\0.jpg': 'buildings',
 'Dataset1\\buildings\\10006.jpg': 'buildings',
 'Dataset1\\buildings\\1001.jpg': 'buildings',
 'Dataset1\\buildings\\10014.jpg': 'buildings',
 'Dataset1\\buildings\\10018.jpg': 'buildings',
 'Dataset1\\buildings\\10029.jpg': 'buildings',
 'Dataset1\\buildings\\10032.jpg': 'buildings',
 'Dataset1\\buildings\\10056.jpg': 'buildings',
 'Dataset1\\buildings\\1009.jpg': 'buildings',
 'Dataset1\\buildings\\10113.jpg': 'buildings',
 'Dataset1\\buildings\\1012.jpg': 'buildings',
 'Dataset1\\buildings\\10126.jpg': 'street',
 'Dataset1\\buildings\\10144.jpg': 'buildings',
 'Dataset1\\buildings\\10151.jpg': 'buildings',
 'Dataset1\\buildings\\10161.jpg': 'buildings',
 'Dataset1\\buildings\\10165.jpg': 'buildings',
 'Dataset1\\buildings\\10176.jpg': 'buildings',
 'Dataset1\\buildings\\10184.jpg': 'buildings',
 'Dataset1\\buildings\\10185.jpg': 'buildings',
 'Dataset1\\buildings\\10191.jpg': 'buildings',
 'Dataset1\\buildings\\10196.jpg': 'buildings',
 '